In [128]:
import pandas as pd
import matplotlib.pyplot as plt 
import re
import textdistance
import numpy as np
%matplotlib inline

In [125]:
# !pip install textdistance

Looking in indexes: http://as-pypi-gcs


In [ ]:
# ee = pd.read_csv('../../data/processed/elevator_to_line_dir_station.csv')

In [452]:
ee = pd.read_csv('../../data/interim/platform_nodes.csv')

In [453]:
ee.head()

,station_name,label,line,direction
0,125 St,A-south,A,south
1,125 St,D-south,D,south
2,125 St,B-south,B,south
3,125 St,C-south,C,south
4,125 St,5-south,5,south


In [429]:
ee[ee.station_name.str.contains('34 St - Penn')]

,station_name,label,line,direction
415,34 St - Penn,C-north,C,north
416,34 St - Penn,E-north,E,north
417,34 St - Penn,A-north,A,north
418,34 St - Penn,A-south,A,south
419,34 St - Penn,C-south,C,south
420,34 St - Penn,E-south,E,south
421,34 St - Penn,1-south,1,south
422,34 St - Penn,2-north,2,north
423,34 St - Penn,2-south,2,south
424,34 St - Penn,3-north,3,north


In [362]:
ee.direction.unique()

array(['south', 'north', 'manhattan', 'norwood 205 st', 'woodlawn',
       'jamaica', 'flushing main st', 'bay ridge', 'coney island',
       'flatbush av brooklyn college',
       'brighton beach, coney island, bay ridge', 'jamaica center',
       'flushing', 'brighton beach', 'wakefield 241 st',
       'coney island, brighton beach', 'jamaica, forest hills',
       'far rockaway'], dtype=object)

In [363]:
routes = pd.read_csv('../../data/raw/google_transit/routes.txt')
trips = pd.read_csv('../../data/raw/google_transit/trips.txt')

In [365]:
trips[trips.route_id == 'W'].head()

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id
16420,W,BFA19GEN-N093-Weekday-00,BFA19GEN-N093-Weekday-00_037300_N..S72R,Whitehall St - South Ferry,1,NaN,NaN
16421,W,BFA19GEN-N093-Weekday-00,BFA19GEN-N093-Weekday-00_039300_N..S72R,Whitehall St - South Ferry,1,NaN,NaN
16422,W,BFA19GEN-N093-Weekday-00,BFA19GEN-N093-Weekday-00_040300_N..S72R,Whitehall St - South Ferry,1,NaN,NaN
16423,W,BFA19GEN-N093-Weekday-00,BFA19GEN-N093-Weekday-00_041300_N..S72R,Whitehall St - South Ferry,1,NaN,NaN
16424,W,BFA19GEN-N093-Weekday-00,BFA19GEN-N093-Weekday-00_042300_N..S72R,Whitehall St - South Ferry,1,NaN,NaN


In [ ]:
'''
Subset routes_wkd to contain line, fuzzy match station name (to upper) with stop_name and name_cur
'''

In [366]:
stops_new = pd.read_csv('../../data/raw/google_transit/stops.txt')

In [367]:
stops_new.head()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
0,101,NaN,Van Cortlandt Park - 242 St,NaN,40.889248,-73.898583,NaN,NaN,1,NaN
1,101N,NaN,Van Cortlandt Park - 242 St,NaN,40.889248,-73.898583,NaN,NaN,0,101
2,101S,NaN,Van Cortlandt Park - 242 St,NaN,40.889248,-73.898583,NaN,NaN,0,101
3,103,NaN,238 St,NaN,40.884667,-73.900870,NaN,NaN,1,NaN
4,103N,NaN,238 St,NaN,40.884667,-73.900870,NaN,NaN,0,103


In [368]:
stops_new[stops_new.stop_name == '30 Av']

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
1314,R04,NaN,30 Av,NaN,40.766779,-73.921479,NaN,NaN,1,NaN
1315,R04N,NaN,30 Av,NaN,40.766779,-73.921479,NaN,NaN,0,R04
1316,R04S,NaN,30 Av,NaN,40.766779,-73.921479,NaN,NaN,0,R04


In [369]:
platforms = stops_new[stops_new.location_type == 0]

In [370]:
platforms.head()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
1,101N,NaN,Van Cortlandt Park - 242 St,NaN,40.889248,-73.898583,NaN,NaN,0,101
2,101S,NaN,Van Cortlandt Park - 242 St,NaN,40.889248,-73.898583,NaN,NaN,0,101
4,103N,NaN,238 St,NaN,40.884667,-73.900870,NaN,NaN,0,103
5,103S,NaN,238 St,NaN,40.884667,-73.900870,NaN,NaN,0,103
7,104N,NaN,231 St,NaN,40.878856,-73.904834,NaN,NaN,0,104


In [371]:
platforms[platforms.stop_name.str.contains('34 St - Penn Station')]

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
76,128N,NaN,34 St - Penn Station,NaN,40.750373,-73.991057,NaN,NaN,0,128
77,128S,NaN,34 St - Penn Station,NaN,40.750373,-73.991057,NaN,NaN,0,128
601,A28N,NaN,34 St - Penn Station,NaN,40.752287,-73.993391,NaN,NaN,0,A28
602,A28S,NaN,34 St - Penn Station,NaN,40.752287,-73.993391,NaN,NaN,0,A28


In [372]:
stop_times = pd.read_csv('../../data/raw/google_transit/stop_times.txt')

In [374]:
stop_times.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,AFA19GEN-1037-Sunday-00_000600_1..S03R,00:06:00,00:06:00,101S,1,NaN,0,0,NaN
1,AFA19GEN-1037-Sunday-00_000600_1..S03R,00:07:30,00:07:30,103S,2,NaN,0,0,NaN
2,AFA19GEN-1037-Sunday-00_000600_1..S03R,00:09:00,00:09:00,104S,3,NaN,0,0,NaN
3,AFA19GEN-1037-Sunday-00_000600_1..S03R,00:10:30,00:10:30,106S,4,NaN,0,0,NaN
4,AFA19GEN-1037-Sunday-00_000600_1..S03R,00:12:00,00:12:00,107S,5,NaN,0,0,NaN


In [375]:
stop_times['line'] = [x[x.find('..')-1:x.find('..')] for x in stop_times.trip_id]

In [376]:
weekday = stop_times[stop_times.trip_id.str.contains('Weekday')]

In [377]:
routes_subset = routes[(routes.route_id.str.len() == 1) & (routes.route_id != 'H')]

In [378]:
trips.head()

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id
0,1,AFA19GEN-1037-Sunday-00,AFA19GEN-1037-Sunday-00_000600_1..S03R,South Ferry,1,NaN,1..S03R
1,1,AFA19GEN-1037-Sunday-00,AFA19GEN-1037-Sunday-00_002600_1..S03R,South Ferry,1,NaN,1..S03R
2,1,AFA19GEN-1037-Sunday-00,AFA19GEN-1037-Sunday-00_004600_1..S03R,South Ferry,1,NaN,1..S03R
3,1,AFA19GEN-1037-Sunday-00,AFA19GEN-1037-Sunday-00_006600_1..S03R,South Ferry,1,NaN,1..S03R
4,1,AFA19GEN-1037-Sunday-00,AFA19GEN-1037-Sunday-00_007200_1..N03R,Van Cortlandt Park - 242 St,0,NaN,1..N03R


In [379]:
weekday = weekday.merge(trips[trips.route_id.isin(routes_subset.route_id)][['trip_id','route_id']],on="trip_id")

In [380]:
unique_stop_ids = pd.DataFrame(weekday.stop_id.unique(),columns=['stop_id'])

In [381]:
unique_stop_ids['routes_wkd'] = [''.join(weekday[weekday.stop_id == x]['route_id'].unique()) for x in unique_stop_ids.stop_id]

In [382]:
platforms = platforms.merge(unique_stop_ids,on='stop_id')

In [412]:
platforms[platforms.stop_name.str.contains('Jackson')]

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,routes_wkd
110,220N,NaN,Jackson Av,NaN,40.816490,-73.907807,NaN,NaN,0,220,25
111,220S,NaN,Jackson Av,NaN,40.816490,-73.907807,NaN,NaN,0,220,25
320,709N,NaN,82 St - Jackson Hts,NaN,40.747659,-73.883697,NaN,NaN,0,709,7
321,709S,NaN,82 St - Jackson Hts,NaN,40.747659,-73.883697,NaN,NaN,0,709,7
342,721N,NaN,Vernon Blvd - Jackson Av,NaN,40.742626,-73.953581,NaN,NaN,0,721,7
343,721S,NaN,Vernon Blvd - Jackson Av,NaN,40.742626,-73.953581,NaN,NaN,0,721,7
652,G14N,NaN,Jackson Hts - Roosevelt Av,NaN,40.746644,-73.891338,NaN,NaN,0,G14,EFMR
653,G14S,NaN,Jackson Hts - Roosevelt Av,NaN,40.746644,-73.891338,NaN,NaN,0,G14,EFMR


In [383]:
platforms.head()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,routes_wkd
0,101N,NaN,Van Cortlandt Park - 242 St,NaN,40.889248,-73.898583,NaN,NaN,0,101,1
1,101S,NaN,Van Cortlandt Park - 242 St,NaN,40.889248,-73.898583,NaN,NaN,0,101,1
2,103N,NaN,238 St,NaN,40.884667,-73.900870,NaN,NaN,0,103,1
3,103S,NaN,238 St,NaN,40.884667,-73.900870,NaN,NaN,0,103,1
4,104N,NaN,231 St,NaN,40.878856,-73.904834,NaN,NaN,0,104,1


In [454]:
ee['possible_stops'] = ''

In [455]:
for index, row in ee.iterrows():
    if row.possible_stops == '':
        subset = platforms[platforms.routes_wkd.str.contains(row.line)]
        if subset.shape[0] > 0:
            subset_stop_names = pd.DataFrame(subset.stop_name.unique(),columns=['stop_name'])
            name_dist = [max(textdistance.jaro_winkler(row.station_name,y),textdistance.jaccard(row.station_name,y)) for y in subset_stop_names.stop_name]
                
            matched_station_name = subset_stop_names.iloc[np.argmax(name_dist),0]
            matched_stop_ids = subset[subset.stop_name == matched_station_name][['stop_id']]
            score = max(name_dist)
            if score > 0.79:
                ee.loc[index,'possible_stops'] = ', '.join(matched_stop_ids.stop_id)
#                 print('{} : {} - {}'.format(row.station_name,matched_station_name,score))


In [267]:
# for index, row in ee.iterrows():
#     if row.stop_ids == '':
#         subset = platforms[platforms.routes_wkd.str.contains(row.line)]
#         if subset.shape[0] > 0:
#             subset_stop_names = pd.DataFrame(subset.stop_name.unique(),columns=['stop_name'])
#             name_dist = [textdistance.jaro_winkler(row.station,y) for y in subset_stop_names.stop_name]
#             matched_station_name = subset_stop_names.iloc[np.argmax(name_dist),0]
#             matched_stop_ids = subset[subset.stop_name == matched_station_name][['stop_id']]
#             score = max(name_dist)
#             if score > 0.5:
# #                 ee.iloc[index,5] = ', '.join(matched_stop_ids.stop_id)
#                 print('{} : {} - {}'.format(row.station,matched_station_name,score))

Broadway-Lafayette/Bleecker St : Brooklyn Bridge - City Hall - 0.535707502374169
Broadway-Lafayette/Bleecker St : Brooklyn Bridge - City Hall - 0.535707502374169
Broadway-Lafayette/Bleecker St : Brooklyn Bridge - City Hall - 0.535707502374169
Broadway-Lafayette/Bleecker St : Brooklyn Bridge - City Hall - 0.535707502374169
Broadway-Lafayette/Bleecker St : Brooklyn Bridge - City Hall - 0.535707502374169
West 4 St - Washington Sq : W 4 St - Wash Sq - 0.79825
West 4 St - Washington Sq : W 4 St - Wash Sq - 0.79825
West 4 St - Washington Sq : W 4 St - Wash Sq - 0.79825
West 4 St - Washington Sq : W 4 St - Wash Sq - 0.79825
West 4 St - Washington Sq : W 4 St - Wash Sq - 0.79825
West 4 St - Washington Sq : W 4 St - Wash Sq - 0.79825
West 4 St - Washington Sq : W 4 St - Wash Sq - 0.79825
West 4 St - Washington Sq : W 4 St - Wash Sq - 0.79825
West 4 St - Washington Sq : W 4 St - Wash Sq - 0.79825
West 4 St - Washington Sq : W 4 St - Wash Sq - 0.79825
West 4 St - Washington Sq : W 4 St - Wash Sq 

In [456]:
ee[ee.possible_stops == ''].shape[0]/ee.shape[0]

0.037122969837587005

In [457]:
ee[ee.possible_stops == '']

,station_name,label,line,direction,possible_stops
7,125 St,METRO_NORTH-south,METRO_NORTH,south,
8,125 St,METRO_NORTH-north,METRO_NORTH,north,
173,Broadway-Lafayette/Bleecker St,6-south,6,south,
181,Broadway-Lafayette/Bleecker St,6-north,6,north,
248,Forest Hills - 71 Av,LIRR-jamaica,LIRR,jamaica,
255,Forest Hills - 71 Av,LIRR-manhattan,LIRR,manhattan,
283,Jackson Hts - Roosevelt Av,7-north,7,north,
284,Jackson Hts - Roosevelt Av,7-south,7,south,
313,Kings Highway,H-brighton beach,H,brighton beach,
315,Kings Highway,C-brighton beach,C,brighton beach,


In [458]:
ee.loc[ee[(ee.possible_stops == '') & (ee.station_name == 'Broadway-Lafayette/Bleecker St')].index,'possible_stops'] = '637N, 637S'
ee.loc[ee[(ee.possible_stops == '') & (ee.station_name == 'Jackson Hts - Roosevelt Av')].index,'possible_stops'] = 'G14N, G14S'
ee.loc[ee[(ee.possible_stops == '') & (ee.station_name == 'Kings Highway')].index,'possible_stops'] = 'D35N, D35S'
ee.loc[ee[(ee.possible_stops == '') & (ee.station_name == 'New Utrecht')].index,'possible_stops'] = 'N04N, N04S'

In [441]:
# ee[(ee.possible_stops == '') & (ee.station_name == 'Broadway-Lafayette/Bleecker St')][['possible_stops']] = '637N, 637S'
# ee[(ee.possible_stops == '') & (ee.station_name == 'Jackson Hts - Roosevelt Av')][['possible_stops']] = 'G14N, G14S'
# ee[(ee.possible_stops == '') & (ee.station_name == 'Kings Highway')][['possible_stops']] = 'D35N, D35S'
# ee[(ee.possible_stops == '') & (ee.station_name == 'New Utrecht')][['possible_stops']] = 'N04N, N04S'

/opt/conda/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/opt/conda/envs/py3/lib/python3.6/site-packages/pandas/core/frame.py:3395: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.loc._setitem_with_indexer((slice(None), indexer), value)
/opt/conda/envs/py3/lib/python3.6/site-packages/pandas/core/frame.py:3367: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/inde

In [459]:
ee[ee.possible_stops == '']

,station_name,label,line,direction,possible_stops
7,125 St,METRO_NORTH-south,METRO_NORTH,south,
8,125 St,METRO_NORTH-north,METRO_NORTH,north,
248,Forest Hills - 71 Av,LIRR-jamaica,LIRR,jamaica,
255,Forest Hills - 71 Av,LIRR-manhattan,LIRR,manhattan,
367,Sutphin Blvd - Archer Av - JFK Airport,LIRR-south,LIRR,south,
368,Sutphin Blvd - Archer Av - JFK Airport,LIRR-north,LIRR,north,
377,Times Sq - 42 St,S-north,S,north,
385,Times Sq - 42 St,S-south,S,south,


In [460]:
ee.head()

,station_name,label,line,direction,possible_stops
0,125 St,A-south,A,south,"A15N, A15S"
1,125 St,D-south,D,south,"A15N, A15S"
2,125 St,B-south,B,south,"A15N, A15S"
3,125 St,C-south,C,south,"A15N, A15S"
4,125 St,5-south,5,south,"621N, 621S"


In [301]:
# pattearn = re.compile('.*N')
# [pattern.match(x) for x in ee.stop_ids[0].split(',')]
# [re.findall('N',x) for x in ee.stop_ids[0].split(',')]

[<_sre.SRE_Match object; span=(0, 4), match='110N'>, None]

In [305]:
# [x for x in ee.stop_ids[0].split(', ') if re.match('.*N$',x)]

['110N']

'S'

In [461]:
def get_dir(df):
#     print(df)
    matches = [x for x in df.possible_stops.split(', ') if re.match('.*{}$'.format(df.direction[0].upper()),x)]
    if matches:
        return matches[0]
    else: 
        return 'NULL'

In [462]:
ee['stop_id'] = ee.apply(get_dir,axis=1)

In [463]:
ee[ee.stop_id == "NULL"]

,station_name,label,line,direction,possible_stops,stop_id
7,125 St,METRO_NORTH-south,METRO_NORTH,south,,NULL
8,125 St,METRO_NORTH-north,METRO_NORTH,north,,NULL
38,149 St - 3 Av,5-manhattan,5,manhattan,"221N, 221S",NULL
39,149 St - 3 Av,2-manhattan,2,manhattan,"221N, 221S",NULL
40,161 St - Yankee Stadium,4-manhattan,4,manhattan,"414N, 414S",NULL
41,161 St - Yankee Stadium,B-manhattan,B,manhattan,"D11N, D11S",NULL
42,161 St - Yankee Stadium,D-manhattan,D,manhattan,"D11N, D11S",NULL
45,161 St - Yankee Stadium,4-woodlawn,4,woodlawn,"414N, 414S",NULL
51,179 St - Jamaica,F-manhattan,F,manhattan,"F01N, F01S",NULL
56,21 St - Queensbridge,F-jamaica,F,jamaica,"B04N, B04S",NULL


In [464]:
ee.to_csv('../../data/processed/platform_to_GTFS_KM.csv')

In [353]:
ee_w_stops = pd.read_csv('../../data/processed/elevator_to_GTFS_KM.csv')

In [355]:
ee_w_stops.shape

(557, 8)

In [357]:
ee_w_stops[ee_w_stops.stop_id.isnull()]

,Unnamed: 0,Unnamed: 0.1,equipment_id,line,direction,station,possible_stops,stop_id
2,2,185,EL122,6,manhattan,Pelham Bay Park,"601N, 601S",NaN
7,7,1464,EL126,METRO-NORTH,south,125 St,NaN,NaN
8,8,1255,EL126,METRO-NORTH,north,125 St,NaN,NaN
12,12,404,EL127,5,wakefield 241 st,Simpson St,"217N, 217S",NaN
13,13,195,EL127,2,wakefield 241 st,Simpson St,"217N, 217S",NaN
14,14,403,EL128,5,manhattan,Simpson St,"217N, 217S",NaN
15,15,194,EL128,2,manhattan,Simpson St,"217N, 217S",NaN
18,18,220,EL130,5,manhattan,149 St - 3 Av,"221N, 221S",NaN
19,19,11,EL130,2,manhattan,149 St - 3 Av,"221N, 221S",NaN
20,20,15,EL132,4,woodlawn,161 St - Yankee Stadium,"414N, 414S",NaN
